# Validating Function Arguments

One possible usage of Pydantic, besides validating data being serialized or deserialized, is to validate the arguments passed to a function.

Admittedely, this is not static type checking - any validation exception will happen at run time, but that's a Python thing, not a Pydantic thing.

Consider this very simplistic example, where we have a function that requires a string argument, and the function requires that the string not be empty (or None).

Here's how we might write this in plain Python:

In [1]:
def extract_first_char(s: str):
    if s is None:
        raise ValueError("argument cannot be None")
    if not isinstance(s, str):
        raise TypeError("argument must be a string")
    if len(s) == 0:
        raise ValueError("argument cannot be an empty string")
    return s[0]

In [2]:
try:
    extract_first_char(100)
except TypeError as ex:
    print(ex)

argument must be a string


In [3]:
try:
    extract_first_char("")
except ValueError as ex:
    print(ex)

argument cannot be an empty string


We could also simplify these validations slightly:

In [4]:
def extract_first_char(s: str):
    if not isinstance(s, str):
        raise TypeError("argument must be a string")
    if not s:
        raise ValueError("argument cannot be an empty string")
    return s[0]

In [5]:
try:
    extract_first_char(100)
except TypeError as ex:
    print(ex)

argument must be a string


In [6]:
try:
    extract_first_char("")
except ValueError as ex:
    print(ex)

argument cannot be an empty string


In [7]:
try:
    extract_first_char(None)
except TypeError as ex:
    print(ex)

argument must be a string


If you think about it, all this validation code we have is done by Pydantic when we define models.

And Pydantic gives us the ability to re-use their validation logic and apply them to function arguments.

Let's first define an annotated type to define what that argument `s` needs to be, beyond just being a string.

In [8]:
from typing import Annotated

from pydantic import Field, ValidationError

In [9]:
NonEmptyString= Annotated[str, Field(min_length=1)] 

Now, this is just a plain annotated type, and we can use it for our function:

In [10]:
def extract_first_char(s: NonEmptyString):
    if s is None:
        raise ValueError("argument cannot be None")
    if not isinstance(s, str):
        raise TypeError("argument must be a string")
    if len(s) == 0:
        raise ValueError("argument cannot be an empty string")
    return s[0]

Of course, this does not change any of our validation code.

We, can however use a special Pydantic decorator that will run the validation of arguments being passed to the function when is being called:

In [11]:
from pydantic import validate_call

In [12]:
@validate_call
def extract_first_char(s: NonEmptyString):
    return s[0]

As you can see, we removed all our validation code, because when we call `extract_first_char()`, it is actually the decorated function, and that decorated function will first run the Pydantic validators before calling our original `extract_first_char` function:

In [13]:
extract_first_char("abc")

'a'

In [14]:
try:
    extract_first_char(None)
except ValidationError as ex:
    print(ex)

1 validation error for extract_first_char
0
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type


In [15]:
try:
    extract_first_char("")
except ValidationError as ex:
    print(ex)

1 validation error for extract_first_char
0
  String should have at least 1 character [type=string_too_short, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/string_too_short


In [16]:
try:
    extract_first_char(100)
except ValidationError as ex:
    print(ex)

1 validation error for extract_first_char
0
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type


Note that the exception is not trapped inside the function, it is trapped in whatever code is calling the function. So you can't use this to modify the behavior of your code inside your function - it is more often used so that you can either take corrective actions prior to calling the function, or present validation exceptions to a user of our app. This can be quite useful in REST APIs for example, where the validation exceptions can simply be reported back as some HTTP error.

Let's look at how we can use a custom validator as well.

In [17]:
from datetime import datetime
from typing import Any

import pytz
from dateutil.parser import parse

def make_utc(dt: datetime) -> datetime:
    print("make_utc called...")
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt
    
def parse_datetime(value: Any):
    print("parse_datetime called...")
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValueError(str(ex))
    return value

In [18]:
from pydantic import BeforeValidator, AfterValidator

DatetimeUTC = Annotated[datetime, BeforeValidator(parse_datetime), AfterValidator(make_utc)]

In [19]:
@validate_call
def func(dt: DatetimeUTC):
    return dt.isoformat()

In [20]:
func("2020/1/1 3pm")

parse_datetime called...
make_utc called...


'2020-01-01T15:00:00+00:00'

As you can see, the validation works just like with model fields - our validators can not only validate, but also modify the data being validated as it goes throiugh the before/after validation pipeline.

So this can be a handy way to not only validate function arguments, but also transform them from their raw input.

A word of caution though, if you not already using Pydantic in your application, think twice about adding it as a dependency just to validate function arguments - maybe it's the correct choice, but it may not be, and for a one-off application, maybe just using plain Python will be more effective:

In [21]:
def func(dt: datetime | str):
    try:
        dt = parse(dt)
    except Exception as ex:
        raise ValueError(str(ex))
    dt = make_utc(dt)
    return dt

In [22]:
func("2020/1/1 3pm")

make_utc called...


datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>)